In [1]:
import os 
from pathlib import Path
import json
from glob import glob

import geopandas as gpd

import matplotlib.pyplot as plt

import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask

In [2]:
# Create paths
data_dir_path = os.path.join(str(Path.home()), "Desktop", "forest_mon", "data")

aoi_path = os.path.join(data_dir_path, "rishiri.json" )

In [3]:
# Load area of interest for clipping
aoi = gpd.read_file(aoi_path)

In [4]:
# Get paths of images to stack
image_paths = sorted(glob(os.path.join(data_dir_path, "L8", "*B?.TIF")))

In [5]:
# Open image and get CRS to convert gdf
with rio.open(image_paths[0]) as sample_image:
    
    # Get EPSG number from string
    image_crs_string = sample_image.crs.data['init'].split(":")[1]

# Re-project gdf into same CRS as data
aoi_reproj = aoi.to_crs(crs=f"epsg:{image_crs_string}")

# Convert coordinates into form that rasterio likes
aoi_reproj_json = [json.loads(aoi_reproj.to_json())['features'][0]['geometry']]

In [17]:
# Clip image and export

for band_path in image_paths:

    # Load image
    with rio.open(band_path) as band:

        # Clip the image
        # out_array is array where pixels outside shapes are masked
        out_array, out_transform = mask(
            dataset=band, shapes=aoi_reproj_json, crop=True)

        # Copy metadata from original file
        out_meta = band.meta.copy()

        # Update metadata with new stuff
        out_meta.update({"driver": "GTiff",
                         "height": out_array.shape[1],
                         "width": out_array.shape[2],
                         "transform": out_transform,
                         "crs": rio.crs.CRS.from_epsg(image_crs_string)})

        # Append "_clipped" to clipped images as output name
        basename_list = os.path.basename(band_path).split(".")
        basename_list[0] = basename_list[0] + "_clipped"
        out_name = '.'.join(basename_list)

        # Export
        with rio.open(os.path.join(data_dir_path, "L8", "clipped", out_name), "w", **out_meta) as dest:
            dest.write(out_array)

In [ ]:
fig, ax = plt.subplots()

show(clipped, ax=ax, cmap='terrain')

In [ ]:
raise ValueError("STOP")

In [ ]:
# Load image
band = rio.open(image_path)

In [ ]:
# Get EPSG number from string
image_crs_string = band.crs.data['init'].split(":")[1]

# Re-project gdf into same CRS as data
aoi_reproj = aoi.to_crs(crs=f"epsg:{image_crs_string}")

In [ ]:
# Clip the image
# out_array is array where pixels outside shapes are masked
out_array, out_transform = mask(dataset=band, shapes=aoi_reproj_json, crop=True)

In [ ]:
# Modify metadata: copy from original file
out_meta = band.meta.copy()

In [ ]:
# Update metadata with new stuff
out_meta.update({"driver": "GTiff",
                 "height": out_array.shape[1],
                 "width": out_array.shape[2],
                 "transform": out_transform,
                 "crs": rio.crs.CRS.from_epsg(image_crs_string)})

In [ ]:
out_tif = "test_clip.tif"

with rio.open(out_tif, "w", **out_meta) as dest:
    dest.write(out_array)

In [ ]:
clipped = rio.open(out_tif)